# ОПИСАНИЕ ПРОЕКТА

Оператор мобильной связи **«Мегалайн»** выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».


**ЦЕЛЬ**: постройть **модель с максимально большим значением `accuracy`**. 

**ЗАДАЧИ**: 
* Довести долю правильных ответов по крайней мере до 0.75. 
* Проверьте accuracy на тестовой выборке самостоятельно.

# ШАГ 1. Открыть файл.

## 1.1 Импортируем необходимые библиотеки и функции

In [180]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import random

## 1.2 Выгрузка файла с проверкой

In [181]:
try:
    data_original = pd.read_csv('users_behavior.csv')
    display(data_original.head())
except:
    data_original = pd.read_csv('/datasets/users_behavior.csv')
    display(data_original.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [182]:
data = data_original

In [183]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [184]:
data.is_ultra.value_counts()/len(data)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

In [185]:
data.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


Можно убрать переменную с количеством звонков, поскольку она сильно коррелирует с количеством минут. Оставляем минуты,поскольку они входят в тариф напрямую, т. е. это более релевантная характеристики оценки интенсивности использования мобиьной связи.

In [186]:
data.drop('calls', axis = 1, inplace = True)
data.head()

,minutes,messages,mb_used,is_ultra
0,311.90,83.0,19915.42,0
1,516.75,56.0,22696.96,0
2,467.66,86.0,21060.45,0
3,745.53,81.0,8437.39,1
4,418.74,1.0,14502.75,0


# ШАГ 2. Разделение данных на обучающую, валидационную и тестовую подвыборки.

In [187]:
# сохраним в переменную псевдорандомное число
rnd_st = 12345

In [188]:
# Поделим наш датасет на части
# df_train = 60%, df_valid и df_test = 20%
# Тем самым наши выборки будут поделены три части 3:1:1
data_train, data_valid = train_test_split(data, 
                                          test_size=0.4, 
                                          random_state=rnd_st)

data_valid, data_test = train_test_split(data_valid, 
                                         test_size=0.5, 
                                         random_state=rnd_st)


После того, как мы поделили наши данные на подвыборки, необходимо посмотреть насколько наши выборки репрезентативны, т. е. похожи на нашу условную генеральную совокукпность. Поскольку отбор в рамках алгоритмов нельзя считать полноценно случайным, то может оказаться, что, например, в обучающую выборку попадут абоненты одного тарифа. В таком случае может возникнуть проблема, когда мы будем обучать машину только на определение абонентов одного тарифа. Вероятность этого крайне мала, но все же стоит проверить.

In [189]:
data.describe().round(2)

,minutes,messages,mb_used,is_ultra
count,3214.00,3214.00,3214.00,3214.00
mean,438.21,38.28,17207.67,0.31
std,234.57,36.15,7570.97,0.46
min,0.00,0.00,0.00,0.00
25%,274.58,9.00,12491.90,0.00
50%,430.60,30.00,16943.24,0.00
75%,571.93,57.00,21424.70,1.00
max,1632.06,224.00,49745.73,1.00


In [190]:
data_train.describe().round(2)

,minutes,messages,mb_used,is_ultra
count,1928.00,1928.00,1928.00,1928.00
mean,436.49,39.21,17007.87,0.31
std,232.47,36.69,7497.74,0.46
min,0.00,0.00,0.00,0.00
25%,275.52,10.00,12497.39,0.00
50%,427.94,31.00,16663.30,0.00
75%,563.80,58.00,21304.78,1.00
max,1390.22,201.00,44296.86,1.00


In [191]:
data_valid.describe().round(2)

,minutes,messages,mb_used,is_ultra
count,643.00,643.00,643.00,643.00
mean,444.10,37.26,17564.26,0.29
std,231.01,35.36,7640.70,0.46
min,0.00,0.00,478.96,0.00
25%,284.15,8.00,12530.80,0.00
50%,442.67,29.00,17222.79,0.00
75%,587.56,57.00,21579.83,1.00
max,1632.06,224.00,49745.73,1.00


In [192]:
data_test.describe().round(2)

,minutes,messages,mb_used,is_ultra
count,643.00,643.00,643.00,643.00
mean,437.46,36.51,17450.18,0.32
std,244.44,35.25,7709.94,0.47
min,0.00,0.00,0.00,0.00
25%,268.48,8.50,12474.04,0.00
50%,424.20,27.00,17284.45,0.00
75%,579.20,55.00,21512.70,1.00
max,1566.45,223.00,45180.75,1.00


**Краткий вывод**:

Наши подвыборки очень похожи друг на друга и на всю нашу условную генеральную совокупность. Следовательно можем переходить к построению моделей.

# ШАГ 3. Исследование различных моделей.

## 3.1 Выявление целевого показателя и характеристик

In [193]:
# Подготовим тренировочную и валидационную выборки
train_features = data_train.drop(['is_ultra'], axis=1)
train_target = data_train['is_ultra']
valid_features = data_valid.drop(['is_ultra'], axis=1)
valid_target = data_valid['is_ultra']

In [194]:
# Напишем функцию, которая расчитывает правильность на валидационной выборке
def valid_accuracy_score(model):
    valid_predictions = model.predict(valid_features)
    return accuracy_score(valid_target, valid_predictions)

## 3.2 Модели `Decision Tree`

In [195]:
def decision_tree_models(features, target, min_depth, max_depth):    
    
    
    data = pd.DataFrame(columns = ['model', 'depth', 'accuracy'])
    
    
    for depth in range(min_depth, max_depth+1):
        model = DecisionTreeClassifier(max_depth=depth, random_state=rnd_st)
        model = model.fit(features, target)
        
        row = pd.DataFrame([[model, depth, valid_accuracy_score(model)]], 
                           columns = ['model', 'depth', 'accuracy'])
        
        
        data = data.append(row, ignore_index=True)

    return data

In [196]:
train_decision_tree_models = decision_tree_models(train_features, train_target, 1, 10)

top_5_decision_tree_models = (train_decision_tree_models
                              .sort_values(by='accuracy', 
                                           ascending = False)
                              .head())

top_5_decision_tree_models['dt_model'] = True
top_5_decision_tree_models

,model,depth,accuracy,dt_model
3,"DecisionTreeClassifier(max_depth=4, random_sta...",4,0.788491,True
4,"DecisionTreeClassifier(max_depth=5, random_sta...",5,0.788491,True
2,"DecisionTreeClassifier(max_depth=3, random_sta...",3,0.785381,True
1,"DecisionTreeClassifier(max_depth=2, random_sta...",2,0.782271,True
8,"DecisionTreeClassifier(max_depth=9, random_sta...",9,0.779160,True


**Краткий вывод**:

* Наилучшей моделью дерева решений оказалась **модель с глубиной дерева равной 4 и 5**.
* Точность пяти лучших моделй из класса дерева решений отличаются друг от друга на уровне десятых долей %. Поэтому сохраним все эти модели для проверки на тестовой выборке.

## 3.3 Модели `Random forest`

In [197]:
def random_forest_tree_models(features, target, min_depth, max_depth, min_estimators, max_estimators):    
    data = pd.DataFrame(columns = ['model', 'depth', 'n_estimators', 'accuracy'])
    for est in range(min_estimators, max_estimators+1):
        for depth in range(min_depth, max_depth+1):
            model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=rnd_st)
            model = model.fit(features, target)
            
            row = pd.DataFrame([[model, depth, est, valid_accuracy_score(model)]], 
                           columns = ['model', 'depth', 'n_estimators', 'accuracy'])
            
            data = data.append(row, ignore_index=True)

                        
    return data

In [198]:
train_random_forest_models = random_forest_tree_models(train_features, train_target, 1, 10, 1, 10)

top_5_random_forest_models = (train_random_forest_models
                              .sort_values(by='accuracy', 
                                           ascending = False)
                              .head())

top_5_random_forest_models['rf_model'] = True
top_5_random_forest_models

,model,depth,n_estimators,accuracy,rf_model
86,"(DecisionTreeClassifier(max_depth=7, max_featu...",7,9,0.796267,True
68,"(DecisionTreeClassifier(max_depth=9, max_featu...",9,7,0.793157,True
96,"(DecisionTreeClassifier(max_depth=7, max_featu...",7,10,0.791602,True
76,"(DecisionTreeClassifier(max_depth=7, max_featu...",7,8,0.791602,True
66,"(DecisionTreeClassifier(max_depth=7, max_featu...",7,7,0.791602,True


**Краткий вывод**:

* Наилучшей моделью случайного леса оказалась модель с 9 деревьями и глубиной деревьев равной 7.
* Точность пяти лучших моделй из класса дерева решений отличаются друг от друга на уровне десятых долей %. Поэтому сохраним все эти модели для проверки на тестовой выборке.

## 3.4 Модель `Logistic regression`

In [199]:
def logistic_regression_model(features, target):
    data = pd.DataFrame(columns = ['model', 'accuracy'])
    model = LogisticRegression(random_state=rnd_st)
    model.fit(features, target)
    row = pd.DataFrame([[model, valid_accuracy_score(model)]], 
                       columns = ['model',  'accuracy'])
    
    data = data.append([row], ignore_index=False)
    return (data)

In [200]:
logistic_regression = logistic_regression_model(train_features, train_target)
logistic_regression['lr_model'] = True
logistic_regression

,model,accuracy,lr_model
0,LogisticRegression(random_state=12345),0.707621,True


**Краткий вывод**:

* Точность логистической регресии значительно уступает моделям из классов дерева решений и случайного леса. Однако сохраним эту модель для проверки ее на тестовой выборке.

## ШАГ 4. Проверка качества моделей на тестовой выборке

In [201]:
top_models = top_5_random_forest_models.append(top_5_decision_tree_models)
top_models = top_models.append(logistic_regression)
top_models.loc[:, ['rf_model', 'dt_model', 'lr_model']] = (top_models
                                                           .loc[:, ['rf_model', 'dt_model', 'lr_model']]
                                                           .fillna(False))
top_models = top_models.sort_values(by='accuracy', ascending = False).reset_index(drop=True)
top_models

,model,depth,n_estimators,accuracy,rf_model,dt_model,lr_model
0,"(DecisionTreeClassifier(max_depth=7, max_featu...",7,9,0.796267,True,False,False
1,"(DecisionTreeClassifier(max_depth=9, max_featu...",9,7,0.793157,True,False,False
2,"(DecisionTreeClassifier(max_depth=7, max_featu...",7,10,0.791602,True,False,False
3,"(DecisionTreeClassifier(max_depth=7, max_featu...",7,8,0.791602,True,False,False
4,"(DecisionTreeClassifier(max_depth=7, max_featu...",7,7,0.791602,True,False,False
5,"DecisionTreeClassifier(max_depth=4, random_sta...",4,NaN,0.788491,False,True,False
6,"DecisionTreeClassifier(max_depth=5, random_sta...",5,NaN,0.788491,False,True,False
7,"DecisionTreeClassifier(max_depth=3, random_sta...",3,NaN,0.785381,False,True,False
8,"DecisionTreeClassifier(max_depth=2, random_sta...",2,NaN,0.782271,False,True,False
9,"DecisionTreeClassifier(max_depth=9, random_sta...",9,NaN,0.779160,False,True,False


In [202]:
test_features = data_test.drop(['is_ultra'], axis=1)
test_target = data_test['is_ultra']

In [203]:
i=0
for model in top_models['model']:
    test_predictions = model.predict(test_features)
    test_accuracy = accuracy_score(test_target, test_predictions)
    top_models.loc[i, 'test_accuracy'] = test_accuracy
    i += 1
    
top_models  

,model,depth,n_estimators,accuracy,rf_model,dt_model,lr_model,test_accuracy
0,"(DecisionTreeClassifier(max_depth=7, max_featu...",7,9,0.796267,True,False,False,0.804044
1,"(DecisionTreeClassifier(max_depth=9, max_featu...",9,7,0.793157,True,False,False,0.804044
2,"(DecisionTreeClassifier(max_depth=7, max_featu...",7,10,0.791602,True,False,False,0.802488
3,"(DecisionTreeClassifier(max_depth=7, max_featu...",7,8,0.791602,True,False,False,0.800933
4,"(DecisionTreeClassifier(max_depth=7, max_featu...",7,7,0.791602,True,False,False,0.800933
5,"DecisionTreeClassifier(max_depth=4, random_sta...",4,NaN,0.788491,False,True,False,0.780715
6,"DecisionTreeClassifier(max_depth=5, random_sta...",5,NaN,0.788491,False,True,False,0.788491
7,"DecisionTreeClassifier(max_depth=3, random_sta...",3,NaN,0.785381,False,True,False,0.779160
8,"DecisionTreeClassifier(max_depth=2, random_sta...",2,NaN,0.782271,False,True,False,0.774495
9,"DecisionTreeClassifier(max_depth=9, random_sta...",9,NaN,0.779160,False,True,False,0.783826


In [204]:
best_model_index = top_models['accuracy'].idxmax()
best_model = top_models.loc[best_model_index, 'model']
print('По точности прогноза на валидационной выборке наилучшей моделью следует считать:')
print(f'{best_model}')

print()

best_model_index = top_models['test_accuracy'].idxmax()
best_model = top_models.loc[best_model_index, 'model']
print('По точности прогноза на тестовой выборке наилучшей моделью следует считать:')
print(f'{best_model}')

print()

best_model_index = top_models.loc[top_models['dt_model'] == True ,'test_accuracy'].idxmax()
best_model = top_models.loc[best_model_index, 'model']
print('По точности прогноза на тестовой выборке наилучшей моделью дерева решений следует считать:')
print(f'{best_model}')

По точности прогноза на валидационной выборке наилучшей моделью следует считать:
RandomForestClassifier(max_depth=7, n_estimators=9, random_state=12345)

По точности прогноза на тестовой выборке наилучшей моделью следует считать:
RandomForestClassifier(max_depth=7, n_estimators=9, random_state=12345)

По точности прогноза на тестовой выборке наилучшей моделью дерева решений следует считать:
DecisionTreeClassifier(max_depth=5, random_state=12345)


**Краткий вывод**:

* Наилучшим классом моделей ожидаемо стал случайный лес.
* Наилучшей моделью с точки зрения точности прогноза **на тестовой выборке** оказалась модель **RandomForestClassifier(max_depth=7, n_estimators=9, random_state=12345)**
* Наилучшей моделью с точки зрения точности прогноза **на валидационной выборке** оказалась модель **RandomForestClassifier(max_depth=7, n_estimators=9, random_state=12345)**
* По точности прогноза **на тестовой выборке** наилучшей моделью **дерева решений** следует считать **DecisionTreeClassifier(max_depth=5, random_state=12345)**
* Модель логистической регрессии показала наихудший результат

ПОДСКАЖИТЕ КАК ПРОВЕРИТЬ А ВМЕНЯЕМОСТЬ. ЧТО ПОНИМАЮТ ПОД ВМЕНЯЕМОСТЬЮ?

In [205]:
#  Наилучшей оказалась модель RandomForestClassifier(max_depth=7, n_estimators=9, random_state=12345)
#  Её и сохраним.
best_model_index = top_models['accuracy'].idxmax()
best_model = top_models.loc[best_model_index, 'model']
best_model

RandomForestClassifier(max_depth=7, n_estimators=9, random_state=12345)

In [206]:
best_model_test_predictions = best_model.predict(test_features)

best_model_test_predictions = pd.Series(best_model_test_predictions, index = test_target.index)

In [207]:
confuse_matrix = confusion_matrix(test_target, best_model_test_predictions)
confuse_matrix

array([[413,  27],
       [ 99, 104]], dtype=int64)

* Из 440 тарифов **`Смарт`** модель верно предсказала 413 наблюдений (27 наблюдений тарифа **`Смарт`** модель определила как `Ультра`)
* Из 203 тарифов **`Ультра`** модель верно предсказала 104 наблюдений (99 наблюдений тарифа **`Ультра`** модель определила как `Смарт`)

In [208]:
precision_score(test_target, best_model_test_predictions)

0.7938931297709924

In [209]:
recall = recall_score(test_target, best_model_test_predictions)
print(r_1_share == 104/(104+99))
r_1_share

True


0.5123152709359606

Создадим случайный процесс определения тарифа. Где вероятности определния тарифов пропорциональны долям этих тарифов в генеральной совокупности.

In [210]:
data['is_ultra'].mean()

0.30647168637212197

In [211]:
random_predictions = []
for i in range(1, len(test_target)+1):
    random_number = random.randint(0, 1000000000000000000)
    if random_number > data['is_ultra'].mean()*100000000000000000:
        random_predictions.append(0)
    else:
        random_predictions.append(1)

In [212]:
random_predictions = pd.Series(random_prediction, index = test_target.index)

### Проверка на вменяемость

In [214]:
random_accuracy = accuracy_score(test_target, random_predictions)
best_model_accuracy = accuracy_score(test_target, best_model_test_predictions)

if best_model_accuracy > random_accuracy:
    print('Наша лучшая модель предсказывает лучше, чем модель случайного выбора тарифа,')
    print('в которой вероятности выбора из двух тарифов пропорциональны их долям в генеральной совокупности.')
else:
    print('Наша лучшая модель предсказывает ХУЖЕ, чем модель случайного выбора тарифа.')
    print('Нужны другие способы моделирования.')

Наша лучшая модель предсказывает лучше, чем модель случайного выбора тарифа,
в которой вероятности выбора из двух тарифов пропорциональны их долям в генеральной совокупности.
